# TCN Experiments

Theodor Marcu
tmarcu@

In [47]:
# Autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# Imports
import numpy as np
import json
import pandas as pd
import numpy as np
import time
import string
from matplotlib import pyplot as plt
import tensorflow as tf
import sys
sys.path.append('../')
from brain2brain import utils
from brain2brain import generators
%matplotlib inline

# TF
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model

# TCN
from brain2brain.tcn import TCN
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense

In [40]:
# Read saved paths for training.
saved_paths_676 = utils.get_file_paths("../brain2brain/train_676_norm_files_projects.txt")

In [41]:
# Split the train files into a training and validation set.
train_676, val_676 = utils.split_file_paths(saved_paths_676, 0.8)

In [42]:
# Print the number of samples.
print(utils.get_total_sample_count(train_676, lookback=512 * 5, length = 1, delay = 512 * 0))

15627


In [43]:
total_electrode_count = 114

In [9]:
# The time we look back.
lookback_window = 512 * 5 # 5 seconds
# Length of sequence predicted.
length_pred = 1 # 1 timestep
# Delay between lookback and length.
delay_pred = 0 # No delay.
# Sampling of electrodes.
samples_per_second = 4 # Samples Per Second
timesteps_per_sample = int(lookback_window // samples_per_second)
# Electrodes
electrode_selection = [0]
electrode_count = len(electrode_selection)

In [25]:
train_676_generator = generators.FGenerator(file_paths = train_676,
                                            lookback=lookback_window, length = length_pred, delay = delay_pred,
                                            batch_size = 128, sample_period = samples_per_second,
                                            electrodes= electrode_selection, shuffle = True, debug=False)

In [26]:
len(train_676_generator)

24

In [27]:
timesteps_per_sample

640

In [28]:
x, y = train_676_generator[0]
x.shape

(128, 640, 1)

In [29]:
len(range(0, 2560, 4))

640

In [33]:
# TCN
i = Input(shape=(timesteps_per_sample, electrode_count), batch_size=128)
m = TCN()(i)
m = Dense(1)(m)

model = Model(inputs=[i], outputs=[m])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(128, 640, 1)]           0         
_________________________________________________________________
tcn_3 (TCN)                  (128, 64)                 120000    
_________________________________________________________________
dense_2 (Dense)              (128, 1)                  65        
Total params: 120,065
Trainable params: 120,065
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile('adam', 'mae')

In [35]:

# config = model.get_config()
# with open('test.json', 'w') as outfile:
#     json.dump(config, outfile)

In [36]:
len(train_676_generator)

24

In [23]:
# Validation Generator
val_676_generator = generators.FGenerator(file_paths = val_676,
                                          lookback=lookback_window, length = length_pred, delay = delay_pred,
                                          batch_size = 128, sample_period = samples_per_second,
                                          electrodes= electrode_selection, shuffle = False, debug=False)

In [24]:
len(val_676_generator)

8

In [22]:
targets = []
for i in range(len(val_676_generator)):
    x, y = val_676_generator[i]
    print(x.shape)


(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)
(128, 640, 1)


In [54]:
import brain2brain.experiments as experiments
experiments.tcn_experiment2()

Running ExperimentExperiment Description: GRU on 1 electrode for patient 676. No activation.saved_paths: ../brain2brain/train_676_norm_files_projects.txtbatch_size: 128lookback_window: 5slength_prediction: 1 timesteps; 0sdelay_pred: 0 timesteps; 0ssamples_per_second: 512timesteps_per_sample: 2560electrode_count: 1
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(128, 2560, 1)]          0         
_________________________________________________________________
tcn_6 (TCN)                  (128, 64)                 120000    
_________________________________________________________________
dense_5 (Dense)              (128, 1)                  65        
Total params: 120,065
Trainable params: 120,065
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
24/24 [==============================] - 102s 4s/step - 

KeyboardInterrupt: 

In [49]:
%pwd

'/home/tmarcu/brain2brain/experiments'

In [130]:
# How many samples to draw from val_676_generator 
# in order to see the entire validation set.
val_steps = len(val_676_generator)

In [ ]:
# model.fit_generator(generator=train_676_generator,
#                     steps_per_epoch=len(train_676_generator),
#                     epochs=20,
#                     validation_data=val_676_generator,
#                     validation_steps=val_steps)

In [13]:
model = load_model("/home/tmarcu/brain2brain/experiments/experiment_test_676_bs128_2_debug/experiment_test_676_bs128_2_debug_model.h5",
                   custom_objects={"TCN": TCN})

In [16]:
train_steps = len(train_676_generator)
val_steps = len(val_676_generator)
p = model.predict_generator(val_676_generator, steps=val_steps,
                            callbacks=None, max_queue_size=100, workers=1,
                            use_multiprocessing=True, verbose=1)

41/41 [==============================] - 44s 1s/step


Process Keras_worker_ForkPoolWorker-2:
Traceback (most recent call last):
  File "/home/tmarcu/.conda/envs/brain2brain_env/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/tmarcu/.conda/envs/brain2brain_env/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tmarcu/.conda/envs/brain2brain_env/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/tmarcu/.conda/envs/brain2brain_env/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 571, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "../brain2brain/generators.py", line 159, in __getitem__
    file_contents = np.load(file_path)
  File "/home/tmarcu/.conda/envs/brain2brain_env/lib/python3.7/site-packages/numpy/lib/npyio.py", line 453, in load
    pickle_kwargs=pickle_kwargs)
  File "/home/tmarcu/.conda/envs/brain2brain_env/li

In [17]:
p

array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]], dtype=float32)

In [62]:
plt.plot(p)
plt.plot(targets)
plt.title('TCN pred on 1 electrode for patient 676.')
plt.legend(['predicted', 'actual'])

array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]], dtype=float32)

In [18]:
targets

NameError: name 'targets' is not defined